# Se crea un modelo de ML para definir caracteristicas como: 'Costo de Combustible', 'Costo Electrico', 'Nivel de Ruido'

# Luego se objetivisa a los 3 tipos de vehiculos: Convencional, Hibrido y Electrico.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier  # O el modelo que prefieras
from sklearn.metrics import accuracy_score  # O métricas como MSE si es regresión
import joblib


# Cargar el dataset
df = pd.read_csv('../Data/costo_operacional_vehiculos_clean.csv')

# Función para categorizar los vehículos
def categorize_vehicle(row):
    if row['Fuel_Type'] in ['Diesel', 'Petrol', 'Petrol/LPG']:
        return 'Convencional'
    elif row['Fuel_Type'] == 'Electricity':
        return 'Eléctrico'
    else:
        return 'Híbrido'

# Aplicar la función al dataframe
df['Vehicle_Type'] = df.apply(categorize_vehicle, axis=1)

# Verificar la distribución
print(df['Vehicle_Type'].value_counts())


# Preprocesamiento (asegúrate de limpiar y preparar los datos para ML)
df = df.dropna()  # Por ejemplo, eliminar valores nulos

# Definir las características (features) y la variable objetivo (target)
X = df[['Fuel_Cost', 'Electric_Cost', 'Noise_Level']]  # Ejemplo de características
y = df['Vehicle_Type']  # Objetivo de predicción (convencional, eléctrico, híbrido)

# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


Vehicle_Type
Convencional    2710
Híbrido          884
Eléctrico        219
Name: count, dtype: int64


In [2]:
# Inicializar el modelo (en este caso, un Random Forest)
model = RandomForestClassifier()

# Entrenar el modelo
model.fit(X_train, y_train)

# Hacer predicciones
y_pred = model.predict(X_test)

# Evaluar el modelo
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy del modelo: {accuracy:.2f}')


Accuracy del modelo: 0.90


In [3]:
# Guardar el modelo en un archivo
joblib.dump(model, 'modelo_rf.joblib')


['modelo_rf.joblib']